## Dataset

https://archive.ics.uci.edu/ml/datasets/pima+indians+diabetes

1. Title: Pima Indians Diabetes Database

2. Sources:
   (a) Original owners: National Institute of Diabetes and Digestive and
                        Kidney Diseases
   (b) Donor of database: Vincent Sigillito (vgs@aplcen.apl.jhu.edu)
                          Research Center, RMI Group Leader
                          Applied Physics Laboratory
                          The Johns Hopkins University
                          Johns Hopkins Road
                          Laurel, MD 20707
                          (301) 953-6231
   (c) Date received: 9 May 1990

3. Past Usage:
    1. Smith,~J.~W., Everhart,~J.~E., Dickson,~W.~C., Knowler,~W.~C., \&
       Johannes,~R.~S. (1988). Using the ADAP learning algorithm to forecast
       the onset of diabetes mellitus.  In {\it Proceedings of the Symposium
       on Computer Applications and Medical Care} (pp. 261--265).  IEEE
       Computer Society Press.

       The diagnostic, binary-valued variable investigated is whether the
       patient shows signs of diabetes according to World Health Organization
       criteria (i.e., if the 2 hour post-load plasma glucose was at least 
       200 mg/dl at any survey  examination or if found during routine medical
       care).   The population lives near Phoenix, Arizona, USA.

       Results: Their ADAP algorithm makes a real-valued prediction between
       0 and 1.  This was transformed into a binary decision using a cutoff of 
       0.448.  Using 576 training instances, the sensitivity and specificity
       of their algorithm was 76% on the remaining 192 instances.

4. Relevant Information:
      Several constraints were placed on the selection of these instances from
      a larger database.  In particular, all patients here are females at
      least 21 years old of Pima Indian heritage.  ADAP is an adaptive learning
      routine that generates and executes digital analogs of perceptron-like
      devices.  It is a unique algorithm; see the paper for details.

5. Number of Instances: 768

6. Number of Attributes: 8 plus class 

    7. For Each Attribute: (all numeric-valued)
       1. Number of times pregnant
       2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
       3. Diastolic blood pressure (mm Hg)
       4. Triceps skin fold thickness (mm)
       5. 2-Hour serum insulin (mu U/ml)
       6. Body mass index (weight in kg/(height in m)^2)
       7. Diabetes pedigree function
       8. Age (years)
       9. Class variable (0 or 1)

8. Missing Attribute Values: Yes

9. Class Distribution: (class value 1 is interpreted as "tested positive for
   diabetes")

   Class Value  Number of instances
   0            500
   1            268

10. Brief statistical analysis:

        Attribute number:    Mean:   Standard Deviation:
        1.                     3.8     3.4
        2.                   120.9    32.0
        3.                    69.1    19.4
        4.                    20.5    16.0
        5.                    79.8   115.2
        6.                    32.0     7.9
        7.                     0.5     0.3
        8.                    33.2    11.8

# Some required packages

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

# Load dataset

In [2]:
data = pd.read_csv('./dataset/pima-indians-diabetes.csv')
data.head(3)

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0


### Add names to columns

In [3]:
columns = [
    'number_pregnant', 
    'glucose_concentration', 
    'blood_pressure', 
    'triceps', 
    'insulin', 
    'bmi', 
    'pedigree', 
    'age', 
    'class'
]

In [4]:
data.columns = columns

In [5]:
data.head()

,number_pregnant,glucose_concentration,blood_pressure,triceps,insulin,bmi,pedigree,age,class
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


# Split features and labels

In [6]:
label_data = data['class'].as_matrix().astype(np.int32)
feat_data = data.drop('class', axis=1).as_matrix().astype(np.float32)

## One-hot encoding

In [7]:
labels = np.zeros((label_data.shape[0], len(set(label_data))))

In [8]:
for i in range(labels.shape[0]):
    labels[i, label_data[i]] = 1

# Split train test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    feat_data,
    labels,
    test_size=0.3,
    random_state=77
)

# Scale the data

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()

In [13]:
scaled_x_train = scaler.fit_transform(X_train)

In [14]:
scaled_x_test = scaler.transform(X_test)

# Manual Architecture

In [15]:
import tensorflow as tf

In [16]:
n_inputs = 8
n_hidden_1 = 13
n_hidden_2 = 10
n_hidden_3 = 13
n_outputs = 2

In [17]:
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
y_true = tf.placeholder(tf.float32, shape=[None, n_outputs])
keep_prob = tf.placeholder(tf.float32)

In [18]:
weights = {
    'W1': tf.Variable(tf.truncated_normal([n_inputs, n_hidden_1], stddev=0.001)),
    'W2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.001)),
    'W3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], stddev=0.001)),
    'out': tf.Variable(tf.truncated_normal([n_hidden_3, n_outputs], stddev=0.001))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_outputs]))
}

In [19]:
layer_1 = tf.nn.sigmoid(tf.matmul(X, weights['W1']) + biases['b1'])
layer_2 = tf.nn.sigmoid(tf.matmul(layer_1, weights['W2']) + biases['b2'])
layer_3 = tf.nn.sigmoid(tf.matmul(layer_2, weights['W3']) + biases['b3'])
y_pred = tf.matmul(layer_3, weights['out']) + biases['out']
y_pred = tf.nn.dropout(y_pred, keep_prob)

In [20]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [21]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)

In [22]:
train = optimizer.minimize(loss)

In [23]:
init = tf.global_variables_initializer()

In [24]:
steps = 5000
dropout = 0.5
batch_sz = 80

In [25]:
from sklearn.utils import shuffle

In [26]:
with tf.Session() as sess:
    sess.run(init)
    
    n_batches = scaled_x_train.shape[0] // batch_sz
    for step in range(steps):
        
        for batch in range(n_batches):
            shuffled_x_train, shuffled_y_train = shuffle(scaled_x_train, y_train)
            
            x_train_batch = shuffled_x_train[batch*batch_sz:(batch*batch_sz+batch_sz), :]
            y_train_batch = shuffled_y_train[batch*batch_sz:(batch*batch_sz+batch_sz), :]
            
            sess.run(train, feed_dict={X: x_train_batch, y_true: y_train_batch, keep_prob:dropout})

            if step % 100 == 0:
                matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
                accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
                acc, lss, preds = sess.run([accuracy, loss, y_pred], feed_dict={X: scaled_x_test, y_true: y_test, keep_prob:dropout})
                print('ON STEM: {:05d}    ACCURACY: {:.5f}    LOSS: {:.5f}'.format(step+1, acc, lss))

ON STEM: 00001    ACCURACY: 0.44589    LOSS: 1.58876
ON STEM: 00001    ACCURACY: 0.41558    LOSS: 1.56591
ON STEM: 00001    ACCURACY: 0.41991    LOSS: 1.55874
ON STEM: 00001    ACCURACY: 0.42424    LOSS: 1.46359
ON STEM: 00001    ACCURACY: 0.41991    LOSS: 1.40928
ON STEM: 00001    ACCURACY: 0.47186    LOSS: 1.30183
ON STEM: 00101    ACCURACY: 0.73160    LOSS: 0.54029
ON STEM: 00101    ACCURACY: 0.74026    LOSS: 0.51357
ON STEM: 00101    ACCURACY: 0.74026    LOSS: 0.56610
ON STEM: 00101    ACCURACY: 0.73593    LOSS: 0.55439
ON STEM: 00101    ACCURACY: 0.75325    LOSS: 0.48963
ON STEM: 00101    ACCURACY: 0.76190    LOSS: 0.53635
ON STEM: 00201    ACCURACY: 0.74026    LOSS: 0.51887
ON STEM: 00201    ACCURACY: 0.80087    LOSS: 0.51154
ON STEM: 00201    ACCURACY: 0.75325    LOSS: 0.52264
ON STEM: 00201    ACCURACY: 0.79221    LOSS: 0.52382
ON STEM: 00201    ACCURACY: 0.79221    LOSS: 0.52770
ON STEM: 00201    ACCURACY: 0.77056    LOSS: 0.53562
ON STEM: 00301    ACCURACY: 0.74892    LOSS: 0

ON STEM: 02501    ACCURACY: 0.73160    LOSS: 0.59458
ON STEM: 02601    ACCURACY: 0.71861    LOSS: 0.62285
ON STEM: 02601    ACCURACY: 0.72727    LOSS: 0.62166
ON STEM: 02601    ACCURACY: 0.72727    LOSS: 0.57455
ON STEM: 02601    ACCURACY: 0.72727    LOSS: 0.51774
ON STEM: 02601    ACCURACY: 0.75758    LOSS: 0.60041
ON STEM: 02601    ACCURACY: 0.74459    LOSS: 0.55560
ON STEM: 02701    ACCURACY: 0.76190    LOSS: 0.56990
ON STEM: 02701    ACCURACY: 0.74459    LOSS: 0.60543
ON STEM: 02701    ACCURACY: 0.76190    LOSS: 0.60848
ON STEM: 02701    ACCURACY: 0.72727    LOSS: 0.64298
ON STEM: 02701    ACCURACY: 0.77489    LOSS: 0.54837
ON STEM: 02701    ACCURACY: 0.73160    LOSS: 0.56806
ON STEM: 02801    ACCURACY: 0.71861    LOSS: 0.67129
ON STEM: 02801    ACCURACY: 0.76623    LOSS: 0.54946
ON STEM: 02801    ACCURACY: 0.74892    LOSS: 0.60204
ON STEM: 02801    ACCURACY: 0.71861    LOSS: 0.66969
ON STEM: 02801    ACCURACY: 0.75758    LOSS: 0.57343
ON STEM: 02801    ACCURACY: 0.71861    LOSS: 0

In [27]:
predictions = np.argmax(preds, axis=1)
y_test = np.argmax(y_test, axis=1)

In [28]:
from sklearn.metrics import confusion_matrix,classification_report

In [29]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.74      0.85      0.79       157
          1       0.53      0.35      0.42        74

avg / total       0.67      0.69      0.67       231

